<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Analisis_SQL_Spark_Datos_Publicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de datasets públicos con PySpark SQL

Este cuaderno permite analizar un dataset público (en formato CSV) utilizando PySpark SQL para ejecutar consultas distribuidas.

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder     .appName("AnalisisDatosPublicosSQL")     .getOrCreate()


## Cargar datos desde archivo CSV
Se utilizará un dataset simulado que representa datos abiertos de conectividad por municipio del Ministerio TIC de Colombia.

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Simulación de datos: municipios y número de conexiones
datos = [
    ("Bogotá", "Internet fijo", 3500000),
    ("Medellín", "Internet fijo", 1200000),
    ("Cali", "Internet móvil", 900000),
    ("Barranquilla", "Internet fijo", 750000),
    ("Cartagena", "Internet móvil", 500000),
    ("Soacha", "Internet fijo", 400000),
    ("Bucaramanga", "Internet móvil", 600000)
]

schema = StructType([
    StructField("municipio", StringType(), True),
    StructField("tipo_servicio", StringType(), True),
    StructField("conexiones", IntegerType(), True)
])

df = spark.createDataFrame(datos, schema)
df.show()


+------------+--------------+----------+
|   municipio| tipo_servicio|conexiones|
+------------+--------------+----------+
|      Bogotá| Internet fijo|   3500000|
|    Medellín| Internet fijo|   1200000|
|        Cali|Internet móvil|    900000|
|Barranquilla| Internet fijo|    750000|
|   Cartagena|Internet móvil|    500000|
|      Soacha| Internet fijo|    400000|
| Bucaramanga|Internet móvil|    600000|
+------------+--------------+----------+



## Consultas SQL distribuidas sobre los datos

In [4]:
# Registrar vista temporal
df.createOrReplaceTempView("conectividad")

# Consultar total de conexiones por tipo de servicio
consulta1 = spark.sql("""
SELECT tipo_servicio, SUM(conexiones) as total_conexiones
FROM conectividad
GROUP BY tipo_servicio
""")
consulta1.show()


+--------------+----------------+
| tipo_servicio|total_conexiones|
+--------------+----------------+
| Internet fijo|         5850000|
|Internet móvil|         2000000|
+--------------+----------------+



In [5]:
# Consultar municipios con más de 800000 conexiones
consulta2 = spark.sql("""
SELECT municipio, conexiones
FROM conectividad
WHERE conexiones > 800000
ORDER BY conexiones DESC
""")
consulta2.show()


+---------+----------+
|municipio|conexiones|
+---------+----------+
|   Bogotá|   3500000|
| Medellín|   1200000|
|     Cali|    900000|
+---------+----------+



## Reflexión sobre el uso de Spark SQL

- Aunque este dataset es pequeño, las mismas operaciones pueden escalar a millones de registros sin modificar el código.
- Spark SQL distribuye las tareas de consulta entre nodos del clúster, lo cual reduce tiempos de espera en escenarios reales.
- La integración entre DataFrames y SQL permite trabajar con datos estructurados de manera flexible, combinando código Python y lenguaje declarativo.

**Recomendación:** Descargue datasets reales de portales abiertos como https://www.datos.gov.co o https://microdatos.dane.gov.co para practicar con volúmenes mayores.